In [ ]:

import os

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    implicit_model_path: Path
    nn_model_path: Path
    scaler_path: Path
    encoders_path: Path
    all_params: dict
    metrics_file_name: Path
    k: int

In [ ]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.HybridRecommender

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            implicit_model_path=config.implicit_model_path,
            nn_model_path=config.nn_model_path,
            scaler_path=config.scaler_path,
            encoders_path=config.encoders_path,
            all_params=params,
            metrics_file_name=config.metrics_file_name,
            k=params.k
        )

In [ ]:
from typing import Dict
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
from src.hybrid_recommender import logger

In [ ]:
class HybridRecommenderEvaluator:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.load_models()
        
    def load_models(self):
        """Load all required models and encoders"""
        self.implicit_model = joblib.load(self.config.implicit_model_path)
        self.nn_model = tf.keras.models.load_model(self.config.nn_model_path)
        self.scaler = joblib.load(self.config.scaler_path)
        encoders = joblib.load(self.config.encoders_path)
        self.user_encoder = encoders['user_encoder']
        self.item_encoder = encoders['item_encoder']
        self.user_decoder = {i: u for u, i in self.user_encoder.items()}
        self.item_decoder = {i: m for m, i in self.item_encoder.items()}

    def evaluate_recommendations(self, test_data: pd.DataFrame) -> Dict[str, float]:
        """Evaluate recommendation quality using standard metrics"""
        # Filter test data to only include known users/items
        test_data = test_data[
            test_data['User-ID'].isin(self.user_encoder) & 
            test_data['ISBN'].isin(self.item_encoder)
        ]
        
        # Group by user and get actual interactions
        user_items = test_data.groupby('User-ID')['ISBN'].apply(set).to_dict()
        
        # Calculate metrics
        precisions = []
        recalls = []
        
        for user_id, actual_items in user_items.items():
            recommended_items = set(self._recommend(user_id))
            relevant_recommended = actual_items & recommended_items
            
            precision = len(relevant_recommended) / len(recommended_items) if recommended_items else 0
            recall = len(relevant_recommended) / len(actual_items) if actual_items else 0
            
            precisions.append(precision)
            recalls.append(recall)
        
        avg_precision = np.mean(precisions)
        avg_recall = np.mean(recalls)
        
        return {
            "precision@k": avg_precision,
            "recall@k": avg_recall,
            "f1_score": 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) 
                        if (avg_precision + avg_recall) > 0 else 0,
            "coverage": self.calculate_coverage(test_data)
        }

    def calculate_coverage(self, test_data: pd.DataFrame) -> float:
        """Calculate what percentage of items can be recommended"""
        all_items = set(self.item_decoder.values())
        recommended_items = set()
        
        for user_id in test_data['User-ID'].unique():
            recommended_items.update(self._recommend(user_id))
        
        return len(recommended_items) / len(all_items)

    def _recommend(self, user_id: int) -> list:
        """Generate recommendations for a single user"""
        user_encoded = self.user_encoder[user_id]
        
        # Get implicit recommendations
        implicit_recs = self.implicit_model.recommend(
            user_encoded, 
            None,  # Passing None since we don't need to retrain
            N=self.config.k*2
        )
        
        # Score with neural network
        user_array = np.array([user_encoded] * len(implicit_recs[0]))
        item_array = np.array(implicit_recs[0])
        
        nn_scores = self.nn_model.predict([user_array, item_array], verbose=0)
        nn_scores = self.scaler.inverse_transform(nn_scores.reshape(-1, 1)).flatten()
        
        # Combine and sort
        combined_scores = implicit_recs[1] * nn_scores
        top_indices = np.argsort(combined_scores)[::-1][:self.config.k]
        
        return [self.item_decoder[item] for item in item_array[top_indices]]

    def save_results(self):
        """Run evaluation and save metrics"""
        test_data = pd.read_csv(self.config.test_data_path)
        metrics = self.evaluate_recommendations(test_data)
        
        # Add model parameters to metrics
        full_results = {
            **metrics,
            "model_parameters": self.config.all_params,
            "num_users": len(self.user_encoder),
            "num_items": len(self.item_encoder)
        }
        
        save_json(path=self.config.metrics_file_name, data=full_results)
        logger.info(f"Evaluation results saved to {self.config.metrics_file_name}")

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    evaluator = HybridRecommenderEvaluator(config=model_evaluation_config)
    evaluator.save_results()
except Exception as e:
    logger.exception("Error during model evaluation")
    raise e